# Problem Set 4

In [ ]:
# Boiler plate

import numpy as np
import cvxopt
import math
import random
cvxopt.solvers.options['show_progress'] = False

prostate_path = r'./prostate_GE.data'
leaf_path = r'./leaf.data'
prostate_data = np.loadtxt(prostate_path, delimiter=",")
leaf_data = np.loadtxt(leaf_path, delimiter=",")

# Problem 1: PCA and Feature Selection

## 1.1 SVMs and PCA 


In [ ]:
# Preprocess data
# label: 2 -> -1

prostate_data[prostate_data[:,-1] == 2, -1] = -1
prostate_train = prostate_data[:80]
prostate_test = prostate_data[80:]

Looking at the eigen values, $k$ should be at most 80 because each data point can be described by a dimension, therefore more than 80 features will not be necessary for strictly describing the training data. However, this will overfit to the training sample. Looking at the top 6 eigen values, it appears that the top 3 values describes the data the most with eigen values $>1500$ and then it greatly decreases beyond the top 3. This implies $k=3$ should describe most of the data and perform adequately. In practice, $k$ should be as large as possible with a trade off between accuracy, generality, and computational cost.

In [ ]:
class PCA():
    def __init__(self):
        self.eigenvalues = None
        self.eigenvectors = None
        self.M = None
    def fit(self, X):
        M = np.mean(X, axis=0)
        W = X - M
        COV = np.matmul(W.T, W)
        self.eigenvalues, self.eigenvectors = np.linalg.eig(COV)
        self.eigenvalues = np.real(self.eigenvalues)
        self.eigenvectors = np.real(self.eigenvectors)
        self.M = M
    def transform(self, X, k):
        return np.matmul(X - self.M, self.eigenvectors[:,:k])

decomp = PCA()
decomp.fit(prostate_train[:, :-1])

for i, ev in enumerate(decomp.eigenvalues[:6]):
    print(f"Rank {i}:\t{ev}")

Rank 0:	21756.602960911554
Rank 1:	1730.7702745518977
Rank 2:	1570.9895436838829
Rank 3:	786.1252951581539
Rank 4:	629.218476538095
Rank 5:	615.1779637357916


* For each k ∈ {1, 2, 3, 4, 5, 6}, project the training data into the best k dimensional subspace
(with respect to the Frobenius norm) and use the SVM with slack formulation to learn a
classifier for each c ∈ {1, 10, 100, 1000}.

In [ ]:
# Note: This directly solves the optimization problem without using kernels
# Using the lagrangian formulation, the exact error differs slightly but 
# the resultant test accuracy and hyperparameter selections are nearly the same

class LinearSlackSVM():
    def __init__(self):
        self.w = 0
        self.b = 0
    def fit(self, X, y, c):
        num_data = X.shape[0] # num of data points
        dim_data = X.shape[1]

        # Compute P
        P = np.identity(dim_data + 1 + num_data)
        for i in range(dim_data, dim_data + 1 + num_data):
            P[i, i] = 0
        P = cvxopt.matrix(P)

        # Compute q
        q = np.zeros(dim_data + 1 + num_data)
        for i in range(dim_data + 1, dim_data + 1 + num_data):
            q[i] = c
        q = cvxopt.matrix(q)

        # Compute G
        G = np.zeros((2 * num_data, dim_data + 1 + num_data))
        for i, row in enumerate(X):
            slack = np.zeros(num_data)
            slack[i] = -1
            temp = np.append(-y[i] * row[:], -y[i])
            temp = np.append(temp, slack)
            G[i] = temp

        for i in range(num_data):
            temp = np.zeros(dim_data + 1 + num_data)
            temp[i + dim_data + 1] = -1
            G[i + num_data] = temp
        G = cvxopt.matrix(G)

        # Compute h
        h = cvxopt.matrix(np.append(-np.ones(num_data), np.zeros(num_data)))

        solution = cvxopt.solvers.qp(P, q, G, h)
        
        w = np.array(solution['x']).flatten()[:dim_data]
        b = np.array(solution['x']).flatten()[dim_data]
        self.w = w
        self.b = b

        return (w, b)

    def predict(self, X):
        return np.sign(np.dot(X, self.w) + self.b)

    def accuracy(self, X, y):
        y_pred = self.predict(X)
        correct = np.equal(y, y_pred)
        return np.count_nonzero(correct) / len(y)

#-------------------------------------------------------------------------------

# Since there are multiple c values that have the highest accuracy
# we will arbitrarily pick the lowest c with the highest acc

# Split training data into 10 folds
folds = np.split(prostate_train, 10)
best_c = 0
best_acc = 0
best_k = 0

c_values = [1, 10, 100, 1000]
k_values = [1, 2, 3, 4, 5, 6]
for c in c_values:
    for k in k_values:
        avg_acc = 0
        for i in range(len(folds)):
            data = [x for j, x in enumerate(folds) if j != i] 
            data = np.concatenate(data)

            X_train = decomp.transform(data[:, :-1], k)
            y_train = data[:, -1]

            X_val = decomp.transform(folds[i][:, :-1], k)
            y_val = folds[i][:, -1]

            model = LinearSlackSVM()
            model.fit(X_train, y_train, c)

            val_acc = model.accuracy(X_val, y_val)
            avg_acc += val_acc

        avg_acc /= len(folds)
        if avg_acc > best_acc:
            best_c = c
            best_acc = avg_acc
            best_k = k
        print(f"c: {c}, k: {k}\t average error: {1 - avg_acc}")

print(f"Best c : {best_c}, best k : {best_k} with error: {1 - best_acc}")

c: 1, k: 1	 average error: 0.36250000000000004
c: 1, k: 2	 average error: 0.25
c: 1, k: 3	 average error: 0.15000000000000002
c: 1, k: 4	 average error: 0.09999999999999998
c: 1, k: 5	 average error: 0.16249999999999998
c: 1, k: 6	 average error: 0.19999999999999996
c: 10, k: 1	 average error: 0.36250000000000004
c: 10, k: 2	 average error: 0.25
c: 10, k: 3	 average error: 0.15000000000000002
c: 10, k: 4	 average error: 0.09999999999999998
c: 10, k: 5	 average error: 0.16249999999999998
c: 10, k: 6	 average error: 0.19999999999999996
c: 100, k: 1	 average error: 0.36250000000000004
c: 100, k: 2	 average error: 0.25
c: 100, k: 3	 average error: 0.15000000000000002
c: 100, k: 4	 average error: 0.09999999999999998
c: 100, k: 5	 average error: 0.16249999999999998
c: 100, k: 6	 average error: 0.19999999999999996
c: 1000, k: 1	 average error: 0.36250000000000004
c: 1000, k: 2	 average error: 0.25
c: 1000, k: 3	 average error: 0.15000000000000002
c: 1000, k: 4	 average error: 0.09999999999999

* What is the performance you achieve on the test set via the proper hyperparameter selection
procedure above?

In [ ]:
# Retrain on all training data
X_train = decomp.transform(prostate_train[:, :-1], best_k)
y_train = prostate_train[:, -1]

model = LinearSlackSVM()
model.fit(X_train, y_train, best_c)

# Test on testing set
X_test = decomp.transform(prostate_test[:, :-1], best_k)
y_test = prostate_test[:, -1]
test_acc = model.accuracy(X_test, y_test)

print(f"Testing accuracy: {test_acc}")

Testing accuracy: 0.7727272727272727


* Now suppose that we don’t do proper hyperparameter selection. What is the best performance
that you can achieve on the test set if you tune the hyperparameters using the test set instead
of the validation set?

In [ ]:
# Since there are multiple c values that have the highest accuracy
# we will arbitrarily pick the lowest c with the highest acc

# Split training data into 10 folds
best_c = 0
best_acc = 0
best_k = 0

c_values = [1, 10, 100, 1000]
k_values = [1, 2, 3, 4, 5, 6]
for c in c_values:
    for k in k_values:
        X_train = decomp.transform(prostate_train[:, :-1], k)
        y_train = prostate_train[:, -1]

        X_test = decomp.transform(prostate_test[:, :-1], k)
        y_test = prostate_test[:, -1]

        model = LinearSlackSVM()
        model.fit(X_train, y_train, c)
        acc = model.accuracy(X_test, y_test)

        if acc > best_acc:
            best_c = c
            best_acc = acc
            best_k = k
        print(f"c: {c}, k: {k}\t error: {1 - acc}")

print(f"Best c : {best_c}, best k : {best_k} with acc: {best_acc}")

c: 1, k: 1	 error: 0.5
c: 1, k: 2	 error: 0.31818181818181823
c: 1, k: 3	 error: 0.2727272727272727
c: 1, k: 4	 error: 0.2272727272727273
c: 1, k: 5	 error: 0.2727272727272727
c: 1, k: 6	 error: 0.2272727272727273
c: 10, k: 1	 error: 0.5
c: 10, k: 2	 error: 0.31818181818181823
c: 10, k: 3	 error: 0.2727272727272727
c: 10, k: 4	 error: 0.2272727272727273
c: 10, k: 5	 error: 0.2727272727272727
c: 10, k: 6	 error: 0.2272727272727273
c: 100, k: 1	 error: 0.5
c: 100, k: 2	 error: 0.31818181818181823
c: 100, k: 3	 error: 0.2727272727272727
c: 100, k: 4	 error: 0.2272727272727273
c: 100, k: 5	 error: 0.2727272727272727
c: 100, k: 6	 error: 0.2272727272727273
c: 1000, k: 1	 error: 0.5
c: 1000, k: 2	 error: 0.31818181818181823
c: 1000, k: 3	 error: 0.2727272727272727
c: 1000, k: 4	 error: 0.2272727272727273
c: 1000, k: 5	 error: 0.2727272727272727
c: 1000, k: 6	 error: 0.2272727272727273
Best c : 1, best k : 4 with acc: 0.7727272727272727


## 1.2 PCA for Feature Selection (25 pts)

1.   Compute the top $k$ eigenvalues and eigenvector of the covariance matrix corresponding to the data matrix omitting the labels (recall that the columns of the data matrix are the input data points). Denote the top $k$ eigenvectors as $v^{(1)}, ... , v^{(k)}$.

2.   Define $\pi_j = \frac{1}{k}\sum_{i=1}^k v_j^{(i)^2}$.

3.   Sample $s$ features independently from the probability distribution defined by $\pi$.

In [ ]:
class PCASelection():
    def __init__(self):
        self.feature_dist = None
        self.feature_select = None
        self.eigenvalues = None
        self.eigenvectors = None
    def fit(self, X):
        # Step 1: Compupte the eigenvectors
        M = np.mean(X, axis=0)
        C = X - M
        COV = np.matmul(C.T, C)
        self.eigenvalues, self.eigenvectors = np.linalg.eig(COV)
        self.eigenvalues = np.real(self.eigenvalues)
        self.eigenvectors = np.real(self.eigenvectors)
        
    def select_features(self, k):
        # Step 2: Create the probability distribution over the features
        self.feature_dist = np.sum(self.eigenvectors[:, :k] ** 2, axis=1) / k
        # Renormalize the distribution due to mathmatical imprecisions
        self.feature_dist = self.feature_dist / np.sum(self.feature_dist)

        # Step 3: sample s features
        s = int(k * math.log(k)) if k > 1 else 1
        self.feature_select = np.random.choice(range(len(self.feature_dist)), size=s, replace=False, p=self.feature_dist)
    def transform(self, X):
        return X[:, self.feature_select]

*  Why does $\pi$ define a probability distribution?

$v^{(i)}_j$ defines the direction of the $i^{th}$ eigenvector with respect to the $j^{th}$ axis. The idea is that this indicates how important $j^{th}$ feature is when computing the linear combination. Because each eigenvectors are normal vectors, the squared sum of each component will add to one. We can therefore add $k$ eigenvectors and normalize it by dividing by $k$ to create a distribution over the original features that estimates how important each feature is when projecting the original data to the $k^{th}$ dimension.

*  Again, using the prostate_GE data set and same procedure as above, for each $k\in\{1,10,20,40,80,160\}$ with $s=\lfloor k\log k\rfloor$, report the average test error of the SVM with slack classifier over $20$ experiments. For each experiment use only the $s$ selected features (note that there may be some duplicates, so only include each feature once). Use the same hyperparameter search for $c$ as in part 1.

In [ ]:
# Since there are multiple c values that have the highest accuracy
# we will arbitrarily pick the lowest c with the highest acc

decomp = PCASelection()
decomp.fit(prostate_train[:,:-1])

# Split training data into 10 folds
folds = np.split(prostate_train, 10)
best_c = 0
best_acc = 0
best_k = 0

c_values = [1, 10, 100, 1000]
k_values = [1, 10, 20, 40, 80, 160]
for c in c_values:
    for k in k_values:
        avg_acc = 0
        # Repeat each hyperparameter pair over 20 CV
        for _ in range(20):
            for i in range(len(folds)):
                data = [x for j, x in enumerate(folds) if j != i] 
                data = np.concatenate(data)

                decomp.select_features(k)

                X_train = decomp.transform(data[:, :-1])
                y_train = data[:, -1]
                X_val = decomp.transform(folds[i][:, :-1])
                y_val = folds[i][:, -1]

                model = LinearSlackSVM()
                model.fit(X_train, y_train, c)
                val_acc = model.accuracy(X_val, y_val)
                avg_acc += val_acc
        avg_acc /= len(folds) * 20
        if avg_acc > best_acc:
            best_c = c
            best_acc = avg_acc
            best_k = k
        print(f"c: {c}, k: {k}\t average error: {1 - avg_acc}")

print(f"Best c : {best_c}, best k : {best_k} with error: {1 - best_acc}")

decomp.select_features(best_k)

# Retrain on all training data
X_train = decomp.transform(prostate_train[:, :-1])
y_train = prostate_train[:, -1]

model = LinearSlackSVM()
model.fit(X_train, y_train, best_c)

# Test on testing set
X_test = decomp.transform(prostate_test[:, :-1])
y_test = prostate_test[:, -1]
test_acc = model.accuracy(X_test, y_test)

print(f"Testing accuracy: {test_acc}")

c: 1, k: 1	 average error: 0.41437500000000005
c: 1, k: 10	 average error: 0.20125000000000004
c: 1, k: 20	 average error: 0.171875
c: 1, k: 40	 average error: 0.17500000000000004
c: 1, k: 80	 average error: 0.12749999999999995
c: 1, k: 160	 average error: 0.114375
c: 10, k: 1	 average error: 0.395625
c: 10, k: 10	 average error: 0.22999999999999998
c: 10, k: 20	 average error: 0.23312500000000003
c: 10, k: 40	 average error: 0.16937500000000005
c: 10, k: 80	 average error: 0.12187499999999996
c: 10, k: 160	 average error: 0.11812500000000004
c: 100, k: 1	 average error: 0.37812500000000004
c: 100, k: 10	 average error: 0.259375
c: 100, k: 20	 average error: 0.218125
c: 100, k: 40	 average error: 0.16437500000000005
c: 100, k: 80	 average error: 0.12437500000000001
c: 100, k: 160	 average error: 0.12312500000000004
c: 1000, k: 1	 average error: 0.39875000000000005
c: 1000, k: 10	 average error: 0.266875
c: 1000, k: 20	 average error: 0.23250000000000004
c: 1000, k: 40	 average error: 0

* Does this provide a reasonable alternative to the SVM with slack formulation without feature
selection on this data set? What are the pros and cons of this approach?

The feature selection does provide a resonable tradeoff between accuracy and computational performance. The pro of this approach is that it drastically reduces the power needed to both train and predict using this model. This may be beneficial in low power situation or when collecting every feature is costly. The con is that the model sacrifices some accuracy. Additionally if the collected data is not representative of real world data, the feature selection may discard features that are able to generalize better.

In [ ]:
# Retrain on all training data
X_train = prostate_train[:, :-1]
y_train = prostate_train[:, -1]

model = LinearSlackSVM()
model.fit(X_train, y_train, best_c)

# Test on testing set
X_test = prostate_test[:, :-1]
y_test = prostate_test[:, -1]
test_acc = model.accuracy(X_test, y_test)

print(f"Testing accuracy: {test_acc}")


Testing accuracy: 0.9090909090909091


# Problem 2: Working with k-means

For this problem, you will use the leaf.data file provided with this problem set. This data set
was generated from the UCI Leaf Data Set

In [ ]:
# Preprocess data
X = leaf_data[:, 1:]
M = np.mean(X, axis = 0)
sd = np.std(X, axis = 0)
X = ((X - M) / sd)

## 2.1 Train a k-means classifier for each $k\in\{10, 15, 20, 25, 30\}$ starting from twenty different random initializations (sample uniformly from $[−3, 3]$ for each attribute) for each $k$. Report the mean and variance of the value of the $k$-means objective obtained for each $k$.

In [ ]:
class KMeans():
    def __init__(self):
        self.centers = None
    def fit(self, X, k):
        # Initalize means
        new_centers = np.random.random_sample((k, X.shape[1])) * 6 - 3
        self.centers = new_centers + 1

        while (np.sum(self.centers - new_centers) != 0):
            self.centers = new_centers
            # Nested list where each list represent all points share the same
            # ith mean
            groups = [[] for _ in range(k)]
            # loop through all points
            for x in X:
                # Calculate distances from current point to means
                distances = [np.linalg.norm(x - center) for center in self.centers]
                # Add point to the corresponding group
                group = np.argmin(distances)
                groups[group].append(x)

            # Calculate new means, if no points are in a group, use prev mean
            new_centers = np.array([np.mean(np.array(group), axis=0) if len(group) != 0 else center for group, center in zip(groups, self.centers)])

        # Calculate objective function
        objective = 0
        for group, center in zip(groups, self.centers):
            for point in group:
                objective += np.linalg.norm(point - center) ** 2
            
        return objective

# ------------------------------------------------------------------------------

k_values = [10, 15, 20, 25, 30]

for k in k_values:
    model = KMeans()
    runs = [model.fit(X, k) for _ in range(20)]
    print(f"k = {k}, mean = {np.mean(runs)}, var = {np.var(runs)}")

k = 10, mean = 1354.7801886138723, var = 74647.95673443752
k = 15, mean = 1110.8436986486881, var = 25612.508374235716
k = 20, mean = 973.3956873904133, var = 36144.8604780891
k = 25, mean = 913.9443494669492, var = 23909.280633463153
k = 30, mean = 850.3265327948357, var = 12049.31990407491


## 2.2 Random initializations can easily get stuck in suboptimal clusterings. An improvement of the k-means algorithm, known as k-means++, instead chooses an initialization as follows:

  (a) Choose a data point uniformly at random to be the first center.

(b) Repeat the following until k centers have been selected:  
i. For each data point x compute the distance between x and the nearest cluster center
in the current set of centers. Denote this distance as dx.

ii. Sample a training data point at random from the distribution $p$ such that $p(x) ∝ d^2_x $
Add the sampled point to the current set of centers.


Repeat the first experiment using this initialization to pick the initial cluster centers for
k-means. Does this procedure result in an improvement? Explain.


In [ ]:
class KMeansplus():
    def __init__(self):
        self.centers = None
    def fit(self, X, k):
        
        # Kmeans++ initlization ----------------

        # list of points that are not means
        points = [row for row in X]
        # list of points that are means
        centers = [points.pop(random.randint(0, len(points)))]

        while (len(centers) != k):
            min_distances_squared = []
            for point in points:
                # calculate the min distance between current point with all centers
                min_distance = min([np.linalg.norm(point - center) for center in centers])
                min_distances_squared.append(min_distance ** 2)

            # Scale the distances squared to create a probability distribution
            min_distances_squared = [dist / sum(min_distances_squared) for dist in min_distances_squared]

            # Sample dist to get next center
            next_center = np.random.choice(range(len(min_distances_squared)), p=min_distances_squared)
            centers.append(points.pop(next_center))

        new_centers = np.array(centers)
        self.centers = new_centers + 1
        # ------------------------

        while (np.sum(self.centers - new_centers) != 0):
            #print(f"dif = {np.sum(self.centers - new_centers)}")
            self.centers = new_centers
            groups = [[] for _ in range(k)]
            for x in X:
                distances = [np.linalg.norm(x - center) for center in self.centers]
                group = np.argmin(distances)
                groups[group].append(x)

            # Calculate new means, if no points are in a group, use prev mean
            new_centers = np.array([np.mean(np.array(group), axis=0) if len(group) != 0 else center for group, center in zip(groups, self.centers)])

        # Calculate objective function
        objective = 0
        for group, center in zip(groups, self.centers):
            for point in group:
                objective += np.linalg.norm(point - center) ** 2
            
        return objective

# ------------------------------------------------------------------------------

k_values = [10, 15, 20, 25, 30]

for k in k_values:
    model = KMeansplus()
    runs = [model.fit(X, k) for _ in range(20)]
    print(f"k = {k}, mean = {np.mean(runs)}, var = {np.var(runs)}")


k = 10, mean = 1046.667471019411, var = 3209.71939637129
k = 15, mean = 758.1106448760277, var = 1127.8378654930525
k = 20, mean = 608.3284704794128, var = 453.2491002451964
k = 25, mean = 526.8292057692009, var = 403.7043940791069
k = 30, mean = 452.3543559247426, var = 279.7483333033425


It seems like kmeans++ did yield an improvement over the original implementation. The mean and variance of the objective is smaller implying that points are more consistently closer to the means. By selecting points through a probability distribution of the min distances, the initalization is more evenly spread out with respect to the data. In constrast, a random uniform initalization, may lead to some center means beinging grouped together while others are far away. This causes some centers to group a majority of the data points while the far away centers will have little to no data points in its group.